In [7]:

from IPython.display import Markdown, display
def in_md(md_txt):
    md_formated_txt = f"--- Response -------<br/>{md_txt}<br/>-------------------"
    display(Markdown(md_formated_txt))

To find supported models for embedding

In [ ]:
from llama_index.embeddings.bedrock import BedrockEmbedding
import json

supported_models = BedrockEmbedding.list_supported_models()
print(json.dumps(supported_models, indent=2))

{
  "amazon": [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-text-v2:0",
    "amazon.titan-embed-g1-text-02"
  ],
  "cohere": [
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
  ]
}


In [21]:
from llama_index.core import get_response_synthesizer
from llama_index.llms.bedrock.base import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import Settings

llm = Bedrock(model="amazon.nova-pro-v1:0"
            , temperature=0
            , max_tokens=3000
            , context_size=4096  # Specify the context window size
)
Settings.llm = llm
# Basic embedding model setup
embed_model = BedrockEmbedding(
    model_name="amazon.titan-embed-text-v2:0"
)
Settings.embed_model = embed_model

/var/folders/8w/ytv2_9yd0nld4gbqjwbs4yw40000gn/T/ipykernel_21650/3653345015.py:6: DeprecationWarning: Call to deprecated class Bedrock. (Should use `llama-index-llms-bedrock-converse` instead, the converse API is the recommended way to use Bedrock.
See: https://docs.llamaindex.ai/en/stable/examples/llm/bedrock_converse/)
  llm = Bedrock(model="amazon.nova-pro-v1:0"
2025-09-13 11:07:35,766 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-13 11:07:37,051 - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [22]:
embed_model.get_text_embedding("hello world")

[-0.02060231938958168,
 0.05661262199282646,
 0.007168131414800882,
 -0.009022098034620285,
 0.039273928850889206,
 -0.02533888816833496,
 0.03170996159315109,
 -0.012241295538842678,
 0.01955483667552471,
 0.013856952078640461,
 -0.006783066783100367,
 -0.009889166802167892,
 0.0004500277864281088,
 -0.08411110192537308,
 0.021822139620780945,
 -0.01758911833167076,
 -0.034111905843019485,
 -0.012357083149254322,
 0.014474267140030861,
 -0.02697339467704296,
 -0.006888084579259157,
 0.05007998272776604,
 -0.007631286978721619,
 -0.01914822869002819,
 -0.04326527565717697,
 -0.032690126448869705,
 0.0022619199007749557,
 -0.02874455787241459,
 0.013076050207018852,
 -0.0004469984269235283,
 0.0779891163110733,
 -0.006882698740810156,
 0.0029081825632601976,
 -0.016296593472361565,
 0.011040322482585907,
 0.05673918128013611,
 0.029598835855722427,
 -0.013738470152020454,
 0.030342038720846176,
 -0.02437487803399563,
 0.05102480202913284,
 0.0008549518533982337,
 -0.004173780791461468,


In [23]:
from llama_index.core import SimpleDirectoryReader
documents =SimpleDirectoryReader(
    input_dir="./data").load_data()

In [24]:
from llama_index.core.node_parser import TokenTextSplitter


splittter = TokenTextSplitter(
    chunk_size= 1024,
    chunk_overlap=20
)
nodes = splittter.get_nodes_from_documents(documents)

In [25]:
import pprint

pprint.pprint(nodes[0], indent=2)

TextNode(id_='58017b01-7712-4e88-9fb7-9615d46777d3', embedding=None, metadata={'file_path': '/Users/ojitha/GitHub/rag_llamaindex/Science_Community/data/AlbertEinstein.txt', 'file_name': 'AlbertEinstein.txt', 'file_type': 'text/plain', 'file_size': 8235, 'creation_date': '2025-09-12', 'last_modified_date': '2025-09-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='35b45bc8-0c8a-4aa8-9b7c-8bfb1b25c542', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/ojitha/GitHub/rag_llamaindex/Science_Community/data/AlbertEinstein.txt', 'file_name': 'AlbertEinstein.txt', 'file_type': 'text/plain', 'file_size': 8235, 'creation_date': '2025-09-12', 'last_modified_date': '2025-09-12'}, hash='ebbeab8f1f

In [26]:
from llama_index.core import VectorStoreIndex


index = VectorStoreIndex(nodes)

In [30]:
q_engine = index.as_query_engine(similarity_top_k = 5) 
response = q_engine.query("who are the scientists in these documents?")
response.response

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [messages] not found, please reformat your input and try again.

In [8]:
in_md(response.response)

--- Response -------<br/>Marie Curie, Pierre Curie, Ernest Rutherford, Henri Poincaré, Paul Langevin, Max Planck, Albert Einstein, Niels Bohr<br/>-------------------

In [9]:
len(response.source_nodes)

5

In [12]:
from llama_index.core import SummaryIndex


summary_index = SummaryIndex(nodes)
s_engine = summary_index.as_query_engine()

In [13]:
summary = s_engine.query("Povide the summary")
in_md(summary.response)

KeyboardInterrupt: 